In [1]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 그래프 기본 테마 설정
# https://coldbrown.co.kr/2023/07/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%A0%84%ED%8E%B8-08-seaborn-sns-set%EC%9D%84-%ED%86%B5%ED%95%B4-%EC%8A%A4%ED%83%80%EC%9D%BC-%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0/
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats

In [2]:
population = pd.read_csv('data/1970-2024 인구데이터.csv', encoding='EUC-KR')
population

,시도별,성별,연령별,시점,추계인구
0,전국,계,계,1970 년,32240827.0
1,전국,계,계,1971 년,32882704.0
2,전국,계,계,1972 년,33505406.0
3,전국,계,계,1973 년,34103149.0
4,전국,계,계,1974 년,34692266.0
...,...,...,...,...,...
58585,제주특별자치도,여자,100세 이상,2020 년,129.0
58586,제주특별자치도,여자,100세 이상,2021 년,145.0
58587,제주특별자치도,여자,100세 이상,2022 년,182.0
58588,제주특별자치도,여자,100세 이상,2023 년,270.0


In [30]:
# 데이터를 보면 데이터와 80-100세까지 나눠진 데이터의 합이 80세 이상의 row에 나타나있음.
# 연도, 연도별 전체인구 수(연령 상관 X), 각 연도별 65세 이상 인구수 = 고령인구 비율 

# 연도별 '전국', '계', '계'에 해당하는 추계인구를 뽑아서 Series로 만들기
national_total = population.query('시도별 == "전국" and 성별 == "계" and 연령별 == "계"')[['시점', '추계인구']]

# 연도를 인덱스로 설정해서 딕셔너리처럼 쓸 수 있게 만듦
national_total_dict = national_total.set_index('시점')['추계인구']

# 원래 population에 새로운 컬럼 추가
population['전국_총인구'] = population['시점'].map(national_total_dict)



# 각 연도별 65세 이상 인구 구하기
age_list = ["65 - 69세", "70 - 74세", "75 - 79세", "80세이상"]

elderly = population.query('시도별 == "전국" and 성별 == "계" and 연령별 in @age_list')
elderly_sum = elderly.groupby('시점')['추계인구'].sum()
population['노년층_인구'] = population['시점'].map(old_year_sum)

population['노년층_비율'] = round((population['노년층_인구'].fillna(0) / population['전국_총인구'].replace(0, np.nan)) * 100, 1)

population


,시도별,성별,연령별,시점,추계인구,전국_총인구,노년층_인구,노년층_비율
0,전국,계,계,1970 년,32240827.0,32240827.0,991308.0,3.1
1,전국,계,계,1971 년,32882704.0,32882704.0,1054523.0,3.2
2,전국,계,계,1972 년,33505406.0,33505406.0,1044658.0,3.1
3,전국,계,계,1973 년,34103149.0,34103149.0,1077254.0,3.2
4,전국,계,계,1974 년,34692266.0,34692266.0,1120510.0,3.2
...,...,...,...,...,...,...,...,...
58585,제주특별자치도,여자,100세 이상,2020 년,129.0,51836239.0,8151867.0,15.7
58586,제주특별자치도,여자,100세 이상,2021 년,145.0,51769539.0,8569865.0,16.6
58587,제주특별자치도,여자,100세 이상,2022 년,182.0,51672569.0,8981133.0,17.4
58588,제주특별자치도,여자,100세 이상,2023 년,270.0,51712619.0,9435816.0,18.2


In [32]:
# 조건 적용: 시도별이 전국이고, 성별이 계, 연령별이 계인 데이터만 추출
filtered = population.query('시도별 == "전국" and 성별 == "계" and 연령별 == "계"')

# 해당 조건 반영된 데이터만 CSV로 저장
filtered.to_csv('전국연도별_노령인구비율.csv', encoding='utf-8-sig', index=False)